In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sudo apt-get install tesseract-ocr

In [ ]:
!pip install pytesseract
!pip install easyocr
!pip install tensorflow

In [4]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt

from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

import xml.etree.ElementTree as ET
import pandas as pd
from glob import glob
import easyocr

In [5]:
path = glob('/content/drive/MyDrive/ANPR/train/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:

    info = ET.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

df = pd.DataFrame(labels_dict)
df.to_csv('/content/drive/MyDrive/ANPR/labels.csv', index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,/content/drive/MyDrive/ANPR/train/Cars108.xml,158,277,216,248
1,/content/drive/MyDrive/ANPR/train/Cars11.xml,131,273,206,234
2,/content/drive/MyDrive/ANPR/train/Cars100.xml,175,214,114,131
3,/content/drive/MyDrive/ANPR/train/Cars101.xml,167,240,202,220
4,/content/drive/MyDrive/ANPR/train/Cars105.xml,152,206,147,159


In [6]:
filename = df['filepath'][0]
def getFilename(filename):
    filename_image = ET.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/content/drive/MyDrive/ANPR/train/',filename_image)
    return filepath_image
getFilename(filename)

'/content/drive/MyDrive/ANPR/train/Cars108.png'

In [7]:
image_path = list(df['filepath'].apply(getFilename))
image_path[:10]#random check

['/content/drive/MyDrive/ANPR/train/Cars108.png',
 '/content/drive/MyDrive/ANPR/train/Cars11.png',
 '/content/drive/MyDrive/ANPR/train/Cars100.png',
 '/content/drive/MyDrive/ANPR/train/Cars101.png',
 '/content/drive/MyDrive/ANPR/train/Cars105.png',
 '/content/drive/MyDrive/ANPR/train/Cars0.png',
 '/content/drive/MyDrive/ANPR/train/Cars106.png',
 '/content/drive/MyDrive/ANPR/train/Cars111.png',
 '/content/drive/MyDrive/ANPR/train/Cars109.png',
 '/content/drive/MyDrive/ANPR/train/Cars107.png']

In [8]:
file_path = image_path[0]
img = cv2.imread(file_path)
img = io.imread(file_path)
fig = px.imshow(img)
fig.update_layout(width=600, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure 1 - Cars108.png with bounding box')
fig.add_shape(type='rect',x0=158, x1=277, y0=216, y1=248, xref='x', yref='y',line_color='cyan')

In [9]:
#Targeting all our values in array selecting all columns
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [10]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [11]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((328, 224, 224, 3), (83, 224, 224, 3), (328, 4), (83, 4))

In [12]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False, input_tensor=Input(shape=(224,224,3)))

headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)

model = Model(inputs=inception_resnet.input,outputs=headmodel)

219055592/219055592 [==============================] - 1s 0us/step


In [ ]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=['accuracy'])
model.summary()

In [14]:
tfb = TensorBoard('object_detection')
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=300,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/300
33/33 [==============================] - 121s 482ms/step - loss: 0.0402 - accuracy: 0.5335 - val_loss: 0.0262 - val_accuracy: 0.7229
Epoch 2/300
33/33 [==============================] - 8s 239ms/step - loss: 0.0134 - accuracy: 0.7226 - val_loss: 0.0193 - val_accuracy: 0.8072
Epoch 3/300
33/33 [==============================] - 8s 235ms/step - loss: 0.0072 - accuracy: 0.8384 - val_loss: 0.0159 - val_accuracy: 0.7470
Epoch 4/300
33/33 [==============================] - 8s 241ms/step - loss: 0.0044 - accuracy: 0.8354 - val_loss: 0.0130 - val_accuracy: 0.7952
Epoch 5/300
33/33 [==============================] - 8s 238ms/step - loss: 0.0034 - accuracy: 0.8323 - val_loss: 0.0128 - val_accuracy: 0.7470
Epoch 6/300
33/33 [==============================] - 8s 242ms/step - loss: 0.0046 - accuracy: 0.8659 - val_loss: 0.0197 - val_accuracy: 0.7590
Epoch 7/300
33/33 [==============================] - 8s 247ms/step - loss: 0.0030 - accuracy: 0.8872 - val_loss: 0.0172 - val_accuracy: 0.80

In [18]:
model.save('/content/drive/MyDrive/ANPR/new_sys_model.keras')

In [19]:
model = tf.keras.models.load_model('/content/drive/MyDrive/ANPR/new_sys_model.keras')
print('Model loaded Sucessfully')

Model loaded Sucessfully


In [37]:
path = '/content/drive/MyDrive/ANPR/test/Cars432.png'
def object_detection(path):

    # Read image
    image = load_img(path) # PIL object
    image = np.array(image,dtype=np.uint8) # 8 bit array (0,255)
    image1 = load_img(path,target_size=(224,224))

    # Data preprocessing
    image_arr_224 = img_to_array(image1)/255.0 # Convert to array & normalized
    h,w,d = image.shape
    test_arr = image_arr_224.reshape(1,224,224,3)

    # Make predictions
    coords = model.predict(test_arr)

    # Denormalize the values
    denorm = np.array([w,w,h,h])
    coords = coords * denorm
    coords = coords.astype(np.int32)

    # Draw bounding on top the image
    xmin, xmax,ymin,ymax = coords[0]
    pt1 =(xmin,ymin)
    pt2 =(xmax,ymax)
    print(pt1, pt2)
    cv2.rectangle(image,pt1,pt2,(0,255,0),3)
    return image, coords

image, cods = object_detection(path)

fig = px.imshow(image)
fig.update_layout(width=700, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure 2')

1/1 [==============================] - 0s 40ms/step
(92, 255) (200, 283)


In [38]:
img = np.array(load_img(path))
xmin ,xmax,ymin,ymax = cods[0]
roi = img[ymin:ymax,xmin:xmax]
fig = px.imshow(roi)
fig.update_layout(width=350, height=250, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure 3 Cropped image')

In [39]:
text = pt.image_to_string(roi)
print(text)

H. DL49 AK49 J



In [40]:
reader = easyocr.Reader(['en'])
result = reader.readtext(roi)

for detection in result:
    print(detection[1])

DL49 AK49
